# <font color='red'>Creating and Customizing a CrewAI Project</font>

## <font color='green'>Table of Contents</font>
1. [Introduction](#Introduction)
2. [Install Required Dependencies](#Install_Required_Dependencies)
3. [Create a New CrewAI Project](#Create_a_New_CrewAI_Project)
4. [Customizing Agent Configuration](#Customizing_Agent_Configuration)
  - [Define Agent Role and Goal](#Define_Agent_Role_and_Goal)
5. [Customizing Task Configuration](#Customizing_Task_Configuration)
  - [Define Task Description_and_Expected Output](#Define_Task_Description_and_Expected_Output)
6. [Implementing the Crew Class](#Implementing_the_Crew_Class)
  - [Define the MyProjectCrew Class and Load Configurations](#Define_the_MyProjectCrew_Class_and_Load_Configurations)
7. [Executing the Project](#Executing_the_Project)
  - [Define the Main Script](#Define_the_Main_Script)
  - [Run the Project](#Run_the_Project)
8. [Conclusion](#Conclusion)

## <font color='green'>1. Introduction</font>

This code will guide you through the process of creating a CrewAI project, customizing agent and task configurations, and running a custom AI workflow.

By the end of this, you'll have a fully functional CrewAI project tailored to your needs.


## <font color='green'>2. Install Required Dependencies</font>

First, we need to install the necessary dependencies, including CrewAI and CrewAI Tools.


In [ ]:
# Install CrewAI and other dependencies
!pip install crewai crewai-tools

## <font color='green'>3. Create a New CrewAI Project</font>

Now, let's create a new CrewAI project named my_project. This project will serve as the foundation for our AI workflow.


In [ ]:
# Create a new CrewAI project
!crewai create crew my_project

In [ ]:
# Change directory to the project (using %cd is fine in Colab)
%cd /content/my_project/src/my_project

###  <font color='blue'>3.1 Set OpenAI Key</font>
We will define the role, goal, and backstory of the researcher agent to specify its responsibilities and expertise.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "Your_OpenAI_Key"

## <font color='green'>4. Customizing Agent Configuration</font>

CrewAI uses agents to perform tasks. In this step, we will customize the agents.yaml file to define a "Job Candidate Researcher" agent responsible for finding potential candidates for a job.

###  <font color='blue'>4.1 Define Agent Role and Goal</font>
We will define the role, goal, and backstory of the researcher agent to specify its responsibilities and expertise.

In [ ]:
# Customizing agents.yaml
agents_yaml = """
researcher:
  role: >
    Job Candidate Researcher
  goal: >
    Find potential candidates for the job
  backstory: >
    You are adept at finding the right candidates by exploring various online
    resources. Your skill in identifying suitable candidates ensures the best
    match for job positions.
"""

# Write to agents.yaml
with open('config/agents.yaml', 'w') as f:
    f.write(agents_yaml)

## <font color='green'>5. Customizing Task Configuration</font>

In this step, we define a task that the researcher agent will perform. The task involves researching and finding potential job candidates.

###  <font color='blue'>5.1 Define Task Description and Expected Output</font>

We will customize the tasks.yaml file to specify the details of the research_candidates_task, including its description, expected output, and the associated agent.


In [ ]:
# Customizing tasks.yaml
tasks_yaml = """
research_candidates_task:
  description: >
    Conduct thorough research to find potential candidates for the specified job.
    Utilize various online resources and databases to gather a comprehensive list of potential candidates.
    Ensure that the candidates meet the job requirements provided.

    Job Requirements:
    {job_requirements}
  expected_output: >
    A list of 10 potential candidates with their contact information and brief profiles highlighting their suitability.
"""

# Write to tasks.yaml
with open('config/tasks.yaml', 'w') as f:
    f.write(tasks_yaml)

## <font color='green'>6. Implementing the Crew Class</font>

The MyProjectCrew class will handle the setup and execution of agents and tasks. We will define the methods to load configurations, create agents, and execute tasks sequentially.

###  <font color='blue'>6.1 Define the MyProjectCrew Class and Load Configurations</font>

The MyProjectCrew class will load the agent and task configurations from the YAML files.

In [ ]:
# Crew.py content
crew_py = """
from crewai import Agent, Crew, Process, Task
from crewai.project import agent, crew, task

class MyProjectCrew:

    def __init__(self, inputs=None):
        self.agents_config = self.load_config('config/agents.yaml')
        self.tasks_config = self.load_config('config/tasks.yaml')
        # Update tasks_config with inputs if provided
        if inputs:
            self.tasks_config['research_candidates_task']['description'] = self.tasks_config['research_candidates_task']['description'].format(**inputs)

    @staticmethod
    def load_config(config_path):
        import yaml
        with open(config_path, 'r') as file:
            return yaml.safe_load(file)

    @agent
    def researcher(self) -> Agent:
        return Agent(
            config=self.agents_config['researcher'],
            verbose=True,
            llm_kwargs={"openai_api_key": "your_api_key_here"}
        )

    @task
    def research_candidates_task(self) -> Task:
        return Task(
            config=self.tasks_config['research_candidates_task'],
            agent=self.researcher()  # Keep the agent
        )  # Remove the context parameter

    @crew
    def crew(self) -> Crew:
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
        )
"""

# Write to crew.py
with open('crew.py', 'w') as f:
    f.write(crew_py)

## <font color='green'>7. Executing the Project</font>

Finally, we define the main script to run the project. This script initializes the MyProjectCrew class with the required job requirements and starts the AI workflow.

###  <font color='blue'>7.1 Define the Main Script</font>

The main script will set up the inputs and kick off the project.

In [ ]:
# Main.py content
main_py = """
from crew import MyProjectCrew

def run():
    inputs = {
        "job_requirements": "Python, Machine Learning, 3+ years of experience"
    }
    MyProjectCrew(inputs).crew().kickoff()

if __name__ == "__main__":
    run()
"""

# Write to main.py
with open('main.py', 'w') as f:
    f.write(main_py)

###  <font color='blue'>7.2 Run the Project</font>

With everything set up, we can now run our project and see the researcher agent in action, finding potential job candidates.

In [ ]:
# Run the main script
!python main.py

## <font color='green'>8. Conclusion</font>

This code walked you through setting up a basic CrewAI project, customizing agent and task configurations, and running the project. You can extend this project by adding more agents, tasks, and tools to suit your needs.